In [2]:
import pandas as pd
import pickle
import random
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from scipy.sparse import hstack,vstack
from sklearn.linear_model import LogisticRegression
import taskALoader as t_a
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC,SVC
from sklearn.utils import class_weight
from sklearn.metrics import classification_report,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV,ShuffleSplit
from sklearn.model_selection import PredefinedSplit
from sklearn.preprocessing import LabelEncoder
from TfidfVectorizerWrapper import TfidfVectorizerWrapper
from ast import literal_eval
import pickle
from sklearn.ensemble import RandomForestClassifier
from statistics import mean,stdev
from textblob import TextBlob
from tqdm import tqdm_notebook as tqdm
from torchMojimaster.examples import example_helper
from torchMojimaster.examples import deepemojisentiment


In [73]:
df = pd.read_csv("task_A_dataset.csv").fillna(' ')
data = t_a.json_creator(df)

X_train_all, X_test = train_test_split(data,test_size=0.2, random_state=1)

X_train, X_val = train_test_split(X_train_all,test_size=0.25, random_state=1)

In [74]:
saved_model_path = "saved_models/"

b000_fn = saved_model_path+'B000_LR_model_basic.sav'
b000_model = pickle.load(open(b000_fn, 'rb'))               #Baseline LogisticRegression
b001_fn = saved_model_path+'B001_SVM_model_basic.sav'
b0001_model = pickle.load(open(b001_fn, 'rb'))           #Baseline SVM

b003_fn = saved_model_path+'605_Lb_logi_binary_clf_model.sav'
b003_model = pickle.load(open(b003_fn, 'rb'))      #manually_annotated binary classifier

e000_fn = saved_model_path+'e007_basic.sav'
e000_model = pickle.load(open(e000_fn, 'rb'))      #manually_annotated binary classifier



#test_path =  "clpsych19_test_data/preprocessed_dataset/"
test_path = "task_A_val_dataset.csv"

'''



b002_fn = 'B000_DEEPEMOj_model.sav'
b002_model = pickle.load(open(b002_fn, 'rb'))           #Sentiment classifier 






p=runBestLogistic(X_train,y_train,X_val,best_pars)
pred_probs.append(p[0].tolist())
'''

"\n\n\n\nb002_fn = 'B000_DEEPEMOj_model.sav'\nb002_model = pickle.load(open(b002_fn, 'rb'))           #Sentiment classifier \n\n\n\n\n\n\np=runBestLogistic(X_train,y_train,X_val,best_pars)\npred_probs.append(p[0].tolist())\n"

In [75]:
#df = pd.read_csv(test_path).fillna(' ')
#data = t_a.json_creator(df)

In [76]:
data = pd.DataFrame(X_val)


df_from_json = df.from_dict(data)
user_postss = list(df_from_json['posts'])

In [77]:
df_from_json.head()

,label,posts,user_id
0,c,"[[9yng4, 1256743121, SuicideWatch, A survivors...",10869
1,d,"[[jvrrt, 1314412249, SuicideWatch, I don't thi...",33604
2,b,"[[35z7ac, 1431631007, SuicideWatch, When does ...",21485
3,a,"[[22ie72, 1396959876, SuicideWatch, /r/gaming ...",2103
4,a,"[[20vl2e, 1395289098, SuicideWatch, My friend ...",48651


In [78]:

all_posts = []
all_posts_bin_clf = []


min_prob = []
max_prob = []
avg_prob = []
std_prob = []

post_count = []
sentiment_prob = []




    
for user in user_postss:
    one_user_posts_bin = []
    one_user_sentiment_bin = []
    one_user_posts = ''
    
    for post_details in user:
        
        #concatenate title and body of a post and put it through a binary clf model
        one_post_bin =  str(post_details[3])+" "+str(post_details[4])
        prob = b003_model.decision_function([one_post_bin])
        one_user_posts_bin.append(prob[0].tolist())
        
        '''
        prob = bin_clf_model.predict_proba([one_post])
        one_user_posts.append(prob[0][1].tolist())
        '''
        sentiment = TextBlob(one_post_bin)
        one_user_sentiment_bin.append(sentiment.sentiment[1])
        
        
        #Concatenate every post from a user
        one_user_posts=one_user_posts+" "+str(post_details[3])+" "+str(post_details[3])+" "+str(post_details[4])
    
    
    
    all_posts_bin_clf.append(one_user_posts_bin)
    sentiment_prob.append(max(one_user_sentiment_bin))
    
    all_posts.append(one_user_posts)
    


In [79]:
for usr_post in all_posts_bin_clf:
    if(len(usr_post)<2):
        std_risk = 0   
    else:
        std_risk = stdev(usr_post)
        
    max_risk = max(usr_post)
    avg_risk = mean(usr_post)
    min_risk = min(usr_post)
    
        
    max_prob.append(max_risk)
    avg_prob.append(avg_risk)
    std_prob.append(std_risk)
    min_prob.append(min_risk)
    post_count.append(len(usr_post))

In [80]:
df_from_json['data'] = all_posts

In [81]:
df_from_json.head()

,label,posts,user_id,data
0,c,"[[9yng4, 1256743121, SuicideWatch, A survivors...",10869,A survivors story A survivors story Truthfull...
1,d,"[[jvrrt, 1314412249, SuicideWatch, I don't thi...",33604,I don't think I can take anymore. [Long] I do...
2,b,"[[35z7ac, 1431631007, SuicideWatch, When does ...",21485,When does it become okay? When does it become...
3,a,"[[22ie72, 1396959876, SuicideWatch, /r/gaming ...",2103,"/r/gaming sent me here, help for a digital fr..."
4,a,"[[20vl2e, 1395289098, SuicideWatch, My friend ...",48651,My friend and roommate told me he tried to co...


In [82]:
test = list(df_from_json['data'])

In [83]:
test[2]

' When does it become okay? When does it become okay? My life is pretty shit right now. When does it become okay to say yes to Suicide?'

In [84]:
lr_probs = []
svm_probs = []

In [85]:
for t in test:

    lr_p = b000_model.predict_proba([t])
    lr_probs.append(lr_p[0].tolist())
  
    
    svm_p = b0001_model.decision_function([t])
    svm_probs.append(svm_p[0].tolist())

In [86]:
svm_probs[0]

[0.0005837254557821553,
 -0.00022896852429215794,
 -0.0005078443157329857,
 0.0001530873842404693]

In [87]:
lr_headers = ['a_prob','b_prob','c_prob','d_prob']
svm_headers = ['svm_a_prob','svm_b_prob','svm_c_prob','svm_d_prob']
df_lr = pd.DataFrame(lr_probs, columns=lr_headers)
df_svm = pd.DataFrame(svm_probs, columns=svm_headers)



# Concatenate LR probs with SVM probs

In [88]:
df_features = pd.concat([df_lr,df_svm],axis=1)

# Include risk probs from annotated model

In [89]:
df_features['max_risk'] = max_prob
df_features['avg_risk'] = avg_prob
df_features['std_risk'] = std_prob

# Include Emotion features

In [90]:
t = ["test sentence"]
df_emotions = deepemojisentiment.get_sentiment_matrix(data,t)

Tokenizing using dictionary from /home/semere/Desktop/CLpsych/torchMojimaster/model/vocabulary.json



[[0.00620417 0.01943046 0.00975873 0.01228339 0.00276523 0.00888439
  0.00804129 0.0112751  0.00298808 0.01034908 0.02149061 0.00182241
  0.01416089 0.00569906 0.00807531 0.01024397 0.03527231 0.00586353
  0.00215316 0.03098154 0.03101745 0.01150388 0.01274652 0.00159728
  0.00928039 0.03171007 0.00619064 0.01677038 0.01322093 0.01662063
  0.0166259  0.01022906 0.01274663 0.01250942 0.01141407 0.02808545
  0.00499765 0.01890216 0.01030899 0.01415732 0.00725454 0.00741935
  0.14245951 0.03248494 0.00677104 0.02870697 0.01944939 0.0069179
  0.00335613 0.00956799 0.00607118 0.01781874 0.0401227  0.01585918
  0.0085838  0.01613444 0.01552611 0.01918738 0.02483873 0.00193355
  0.00203777 0.00313395 0.04182546 0.00416176]]


In [93]:
df_features = pd.concat([df_features,df_emotions],axis=1)

In [94]:
df_features.head()

,a_prob,b_prob,c_prob,d_prob,svm_a_prob,svm_b_prob,svm_c_prob,svm_d_prob,max_risk,avg_risk,...,E182,E183,E184,E185,E186,E187,E188,E189,E190,E191
0,0.282073,0.206163,0.214389,0.297374,0.000584,-0.000229,-0.000508,0.000153,0.911406,-0.704142,...,0.006319,0.020073,0.002163,0.013148,0.021429,0.011327,0.011665,0.014493,0.004749,0.019667
1,0.226823,0.174442,0.280987,0.317749,0.000182,-0.000241,-0.000075,0.000134,0.864240,0.864240,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.299986,0.181824,0.282004,0.236186,0.000485,-0.000368,0.000034,-0.000150,0.257642,0.257642,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.435448,0.140453,0.222872,0.201227,0.001242,-0.000621,-0.000311,-0.000310,-0.572106,-0.572106,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.274052,0.168468,0.228554,0.328926,0.000405,-0.000394,-0.000258,0.000246,0.288580,0.288580,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [72]:
#df_features.to_csv("ensemble_validation.csv", encoding='utf-8', index=False)
features = df_features.columns[0:]
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_features[features] = scaler.fit_transform(df_features[features])
features

Index(['a_prob', 'b_prob', 'c_prob', 'd_prob', 'svm_a_prob', 'svm_b_prob',
       'svm_c_prob', 'svm_d_prob', 'max_risk', 'avg_risk',
       ...
       'E182', 'E183', 'E184', 'E185', 'E186', 'E187', 'E188', 'E189', 'E190',
       'E191'],
      dtype='object', length=203)

In [71]:
df_features.head()

,a_prob,b_prob,c_prob,d_prob,svm_a_prob,svm_b_prob,svm_c_prob,svm_d_prob,max_risk,avg_risk,...,E182,E183,E184,E185,E186,E187,E188,E189,E190,E191
0,0.433858,0.527313,-1.246793,-0.050134,0.870832,-0.190849,-1.789796,-0.004820,0.269066,-1.343831,...,2.248132,2.443843,-0.032526,3.106197,5.294167,2.268735,2.040027,3.399076,0.759940,2.069808
1,-0.203238,-0.476620,0.394425,0.226346,0.121126,-0.233735,0.088737,-0.055038,0.213961,0.454241,...,-0.424162,-0.463499,-0.406786,-0.371234,-0.372811,-0.376438,-0.326034,-0.420380,-0.451419,-0.291969
2,0.640407,-0.242973,0.419491,-0.880437,0.686997,-0.698236,0.559331,-0.795235,-0.494740,-0.241194,...,-0.424162,-0.463499,-0.406786,-0.371234,-0.372811,-0.376438,-0.326034,-0.420380,-0.451419,-0.291969
3,2.202418,-1.552291,-1.037734,-1.354821,2.098366,-1.616814,-0.934890,-1.211232,-1.464152,-1.192459,...,-0.424162,-0.463499,-0.406786,-0.371234,-0.372811,-0.376438,-0.326034,-0.420380,-0.451419,-0.291969
4,0.341364,-0.665675,-0.897730,0.378023,0.537503,-0.790207,-0.704870,0.238553,-0.458594,-0.205724,...,-0.424162,-0.463499,-0.406786,-0.371234,-0.372811,-0.376438,-0.326034,-0.420380,-0.451419,-0.291969


In [386]:
le = preprocessing.LabelEncoder()
test_labels = list(df_from_json['label'])

print(test_labels)

test_labels=le.fit_transform(test_labels)


['c', 'd', 'b', 'a', 'a', 'd', 'a', 'a', 'b', 'c', 'd', 'a', 'c', 'd', 'd', 'd', 'd', 'd', 'd', 'd', 'b', 'd', 'd', 'c', 'd', 'c', 'a', 'a', 'a', 'c', 'b', 'd', 'd', 'a', 'a', 'd', 'd', 'd', 'c', 'a', 'c', 'b', 'a', 'a', 'a', 'd', 'd', 'a', 'd', 'd', 'd', 'c', 'd', 'b', 'd', 'a', 'd', 'a', 'd', 'c', 'd', 'a', 'c', 'a', 'd', 'd', 'a', 'a', 'd', 'a', 'd', 'd', 'b', 'b', 'b', 'c', 'b', 'd', 'd', 'd', 'd', 'd', 'c', 'a', 'c', 'd', 'c', 'a', 'c', 'a', 'a', 'c', 'a', 'b', 'd', 'a', 'a', 'd', 'd']


In [387]:
test_labels

array([2, 3, 1, 0, 0, 3, 0, 0, 1, 2, 3, 0, 2, 3, 3, 3, 3, 3, 3, 3, 1, 3,
       3, 2, 3, 2, 0, 0, 0, 2, 1, 3, 3, 0, 0, 3, 3, 3, 2, 0, 2, 1, 0, 0,
       0, 3, 3, 0, 3, 3, 3, 2, 3, 1, 3, 0, 3, 0, 3, 2, 3, 0, 2, 0, 3, 3,
       0, 0, 3, 0, 3, 3, 1, 1, 1, 2, 1, 3, 3, 3, 3, 3, 2, 0, 2, 3, 2, 0,
       2, 0, 0, 2, 0, 1, 3, 0, 0, 3, 3])

In [388]:
clf = e000_model
predicted=clf.predict(df_features)
print(classification_report(test_labels, predicted))
print ("Accuracy: {}".format(accuracy_score(test_labels, predicted)))
predicted

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        29
           1       0.10      0.73      0.18        11
           2       0.11      0.12      0.11        17
           3       0.50      0.02      0.05        42

   micro avg       0.11      0.11      0.11        99
   macro avg       0.18      0.22      0.08        99
weighted avg       0.24      0.11      0.06        99

Accuracy: 0.1111111111111111


array([1, 1, 1, 2, 1, 1, 1, 1, 1, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 3, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 3,
       2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
       1, 1, 2, 2, 2, 1, 1, 2, 1, 1, 1])